## Init Database

In [6]:
import os
import sqlite3

# ============================================
# Configuration
# ============================================

# Path to the SQLite DB
DATABASE_PATH = os.path.join("/Users/greyson/Projects/custom_gallery/gallery/prisma", 'image_analysis.db')

# Initialize the database (only run once)
if not os.path.exists(DATABASE_PATH):
        initialize_database()
else:
    print("Database already exists.")
        
# ============================================
# Database Initialization
# ============================================

def initialize_database():
    """
    Initializes the SQLite database with the required tables.
    Creates 'Posts' and 'Images' tables along with related tables for image analysis.
    """
    conn = sqlite3.connect(DATABASE_PATH)
    cursor = conn.cursor()
    
    # Enable foreign key support
    cursor.execute('PRAGMA foreign_keys = ON;')
    
    # Create Posts table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Posts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        shortcode TEXT UNIQUE,
        username TEXT,
        caption TEXT,
        post_date TEXT
    )
    ''')
    
    # Create Images table with foreign keys to Posts
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Images (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        absolute_file_path TEXT UNIQUE,
        relative_file_path TEXT UNIQUE,
        filename TEXT,
        processed_at TEXT,
        post_id INTEGER,
        FOREIGN KEY(post_id) REFERENCES Posts(id) ON DELETE CASCADE
    )
    ''')
    
    # Create Luminance table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Luminance (
        image_id INTEGER PRIMARY KEY,
        mean_luminance REAL,
        median_luminance REAL,
        std_luminance REAL,
        dynamic_range REAL,
        rms_contrast REAL,
        michelson_contrast REAL,
        luminance_skewness REAL,
        luminance_kurtosis REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')
    
    # Create Saturation table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Saturation (
        image_id INTEGER PRIMARY KEY,
        mean_saturation REAL,
        median_saturation REAL,
        std_saturation REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')
    
    # Create GLCM table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS GLCM (
        image_id INTEGER PRIMARY KEY,
        contrast REAL,
        correlation REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')
    
    # Create Laplacian table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Laplacian (
        image_id INTEGER PRIMARY KEY,
        variance REAL,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')
    
    # Create KMeansClustering table with 'id' as PRIMARY KEY
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS KMeansClustering (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        image_id INTEGER,
        num_clusters INTEGER,
        FOREIGN KEY(image_id) REFERENCES Images(id) ON DELETE CASCADE
    )
    ''')
    
    # Create Clusters table referencing KMeansClustering.id
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Clusters (
        clustering_id INTEGER,
        cluster_index INTEGER,
        r INTEGER,
        g INTEGER,
        b INTEGER,
        count INTEGER,
        percentage REAL,
        FOREIGN KEY(clustering_id) REFERENCES KMeansClustering(id) ON DELETE CASCADE,
        PRIMARY KEY (clustering_id, cluster_index)
    )
    ''')
    
    conn.commit()
    conn.close()
    print("Database initialized successfully.")

def connect_db():
    """
    Connects to the SQLite database with foreign key support enabled.
    
    Returns:
        sqlite3.Connection: The database connection object.
    """
    conn = sqlite3.connect(DATABASE_PATH)
    conn.execute('PRAGMA foreign_keys = ON;')  # Enable foreign key support
    return conn

Database initialized successfully.


## Download IG Posts

In [7]:
import os
import sqlite3
import instaloader
import time
from datetime import timedelta
from datetime import datetime

# Create an Instaloader instance with custom directory and filename patterns
L = instaloader.Instaloader(
    dirname_pattern='../gallery/public/img/{target}/{date_utc:%Y-%m-%d_%H-%M-%S}_{shortcode}',
    filename_pattern='{filename}',
    download_videos=False,             # Skip downloading videos
    download_video_thumbnails=False    # Skip downloading video thumbnails
)

# Replace 'instagram_username' with the actual Instagram username
username = 'greyson.color'  # e.g., 'nasa'

# Get the profile metadata
profile = instaloader.Profile.from_username(L.context, username)

# Fetch all posts from the profile
posts = list(profile.get_posts())  # Convert to list to calculate length

total_posts = len(posts)  # Get total number of posts
print(f"Total posts to download: {total_posts}")

# Track progress and download time
start_time = time.time()  # Track the time when the download starts

# Connect to the database
conn = connect_db()
cursor = conn.cursor()

for index, post in enumerate(posts, start=1):
    # Extract post data
    shortcode = post.shortcode
    post_date = post.date_utc.strftime('%Y-%m-%d %H:%M:%S')  # Format date as string
    caption = post.caption if post.caption else ""
    post_username = post.owner_username

    # Insert or ignore the post into Posts table
    cursor.execute('''
    INSERT OR IGNORE INTO Posts (shortcode, username, caption, post_date)
    VALUES (?, ?, ?, ?)
    ''', (shortcode, post_username, caption, post_date))

    # Retrieve the post ID
    cursor.execute('SELECT id FROM Posts WHERE shortcode = ?', (shortcode,))
    post_id = cursor.fetchone()[0]

    # Set the directory for the post
    dirname = f"../gallery/public/img/{username}/{post.date_utc:%Y-%m-%d_%H-%M-%S}_{post.shortcode}"
    L.dirname_pattern = dirname

    # Start the timer for this post
    post_start_time = time.time()

    # Download the post (images only)
    L.download_post(post, target='')

    # End the timer for this post
    post_end_time = time.time()

    # Calculate time taken for the current post
    time_for_post = post_end_time - post_start_time

    # Calculate total elapsed time so far
    elapsed_time = time.time() - start_time

    # Estimate remaining time based on the average time per post
    avg_time_per_post = elapsed_time / index
    remaining_posts = total_posts - index
    estimated_remaining_time = avg_time_per_post * remaining_posts

    # List the image files in 'dirname'
    image_files = []
    for root, dirs, files in os.walk(dirname):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
                image_files.append(os.path.join(root, file))

    # Insert image data into Images table for each image
    for image_file in image_files:
        # Absolute file path
        absolute_file_path = os.path.abspath(image_file)
        # Relative file path (after 'public')
        try:
            public_index = absolute_file_path.index('public') + len('public')
            relative_file_path = absolute_file_path[public_index:].replace('\\', '/')
        except ValueError:
            # 'public' not in path, handle accordingly
            print(f"'public' not found in the path: {absolute_file_path}")
            relative_file_path = f"/img/{username}/{post.date_utc:%Y-%m-%d_%H-%M-%S}_{post.shortcode}/{os.path.basename(image_file)}"
        # Prepend '/' to make it an absolute URL path
        relative_file_path = f"/{relative_file_path.lstrip('/')}"
        # Filename
        image_filename = os.path.basename(image_file)
        # Processed at timestamp
        processed_at = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        # Insert into Images table
        cursor.execute('''
        INSERT OR IGNORE INTO Images (absolute_file_path, relative_file_path, filename, processed_at, post_id)
        VALUES (?, ?, ?, ?, ?)
        ''', (absolute_file_path, relative_file_path, image_filename, processed_at, post_id))

    conn.commit()

    # Print progress
    print(f"Downloaded {index}/{total_posts} posts.")
    print(f"Time for last post: {time_for_post:.2f} seconds.")
    print(f"Estimated remaining time: {timedelta(seconds=int(estimated_remaining_time))}")
    print("-------------------------------------------------------")

# Close the database connection
conn.close()

# Calculate total time spent after all posts are downloaded
total_time_spent = time.time() - start_time
print(f"All {total_posts} posts downloaded.")
print(f"Total time spent: {timedelta(seconds=int(total_time_spent))}")


Total posts to download: 2
../gallery/public/img/greyson.color/2023-09-20_18-51-02_CxbKCv8PS3k/381237906_1517126165781046_8021449951689993247_n.jpg ../gallery/public/img/greyson.color/2023-09-20_18-51-02_CxbKCv8PS3k/381136614_3522798227968195_4326888869906999098_n.jpg ../gallery/public/img/greyson.color/2023-09-20_18-51-02_CxbKCv8PS3k/380737180_825114379087528_2830082864892029683_n.jpg [🌞 OUTSIDE ALL SUMMER 🌞 by the…] json 
Downloaded 1/2 posts.
Time for last post: 1.15 seconds.
Estimated remaining time: 0:00:01
-------------------------------------------------------
../gallery/public/img/greyson.color/2023-05-29_22-01-39_Cs19PKES5MU/350238228_997404024767128_304389176336615097_n.jpg 

/var/folders/mh/bv4xg_ks04v6451j4dcrxz840000gn/T/ipykernel_5746/263907748.py:100: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  processed_at = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')


../gallery/public/img/greyson.color/2023-05-29_22-01-39_Cs19PKES5MU/350087240_559904122975610_4195451578829777854_n.jpg [Last week I had the incredibl…] json 
Downloaded 2/2 posts.
Time for last post: 0.47 seconds.
Estimated remaining time: 0:00:00
-------------------------------------------------------
All 2 posts downloaded.
Total time spent: 0:00:01
